In [ ]:
!pip -q install langchain huggingface_hub transformers sentence_transformers 

In [ ]:
# !pip install accelerate 
# !pip install --upgrade accelerate
!pip install --upgrade transformers
#bitsandbytes
# !pip install -i https://test.pypi.org/simple/ bitsandbytes


In [2]:
!nvidia-smi

Sat May  6 16:52:01 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 526.47       Driver Version: 526.47       CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name            TCC/WDDM | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ... WDDM  | 00000000:01:00.0  On |                  N/A |
|  0%   47C    P8     9W / 170W |    770MiB / 12288MiB |      8%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [8]:
## Local model

from langchain.llms import HuggingFacePipeline
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline, AutoModelForSeq2SeqLM
from accelerate import init_empty_weights


# with init_empty_weights():
#     model = AutoModelForCausalLM.from_config(config)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_id = 'facebook/blenderbot-1B-distill' #'google/flan-t5-large' 
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSeq2SeqLM.from_pretrained(model_id) #, torch_dtype=torch.float16, load_in_8bit=False, device_map='auto')
model.to(device)

pipe = pipeline(
    "text2text-generation",
    model=model, 
    tokenizer=tokenizer, 
    max_length=100
)

local_llm = HuggingFacePipeline(pipeline=pipe)


In [ ]:
print(local_llm('What is the capital of France? '))

paris


In [ ]:
from langchain import PromptTemplate, HuggingFaceHub, LLMChain

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

llm_chain = LLMChain(prompt=prompt, 
                     llm=local_llm
                     )

question = "Code for giving full permission with chmod" #"What is the capital of England?"

print(llm_chain.run(question))

chmod is a command line utility that allows you to change the permissions of a file. The answer: chmod.
